In [29]:
from mckit import *

### Read MCNP input file

read_mcnp function reads MCNP input file and returns Universe instance. Universe instance contains _cell variable, list of cells. 

In [30]:
model1 = read_mcnp('test1.i')
type(model1)

mckit.universe.Universe

In [31]:
model1.name()

0

It is possible to iterate over universe entities. Entities of inner universes are not included.

In [32]:
for cell in model1:
    print(cell.mcnp_repr())

1 0 -4 -2 1 -3 5 -6
        IMP:N=1.000 
        U=0 FILL=1 
2 0 3:-1:2
        IMP:N=0.000 
        U=0 


#### Get all universes

In [33]:
universes = model1.get_universes()
udict = {}
for u in universes:
    udict[u.name()] = u
    print('name: ', u.name(), 'The number of cells: ', len(u))

name:  1 The number of cells:  3
name:  0 The number of cells:  2


In [7]:
cell_1 = udict[0]._cells[0]
print(cell_1.mcnp_repr())

1 0 -4 -2 1 -3 5 -6
        IMP:N=1.000 
        U=0 FILL=1 


In [8]:
surfaces = cell_1.shape.get_surfaces()
for s in surfaces:
    print(s.mcnp_repr())

6 PZ 30
2 CZ 5
5 PZ -20
4 CZ 10
1 PZ -10
3 PZ 20


Surfaces 4, 5 and 6 are redundant. To simplify cell description *simplify* method can be used.

In [34]:
print(cell_1.simplify.__doc__)

Simplifies this cell by removing unnecessary surfaces.

        The simplification procedure goes in the following way.
        # TODO: insert brief description!

        Parameters
        ----------
        box : Box
            Box where geometry should be simplified.
        split_disjoint : bool
            Whether to split disjoint geometries into separate geometries.
        min_volume : float
            The smallest value of box's volume when the process of box splitting
            must be stopped.
        trim_size : int
            Max size of set to return. It is used to prevent unlimited growth
            of the variant set.

        Returns
        -------
        simple_cell : Cell
            Simplified version of this cell.
        


In [35]:
cell_1s = cell_1.simplify(min_volume=1.e-3)
print(cell_1s.mcnp_repr())

1 0 -2 1 -3
        IMP:N=1.000 
        U=0 FILL=1 


To flatten mcnp model *apply_fill* method of Universe can be used. It modifies current universe: inserts all cells of inner universes. But *simplify* method must be called separately.

In [36]:
for c in model1:
    print(c.mcnp_repr())

1 0 -4 -2 1 -3 5 -6
        IMP:N=1.000 
        U=0 FILL=1 
2 0 3:-1:2
        IMP:N=0.000 
        U=0 


In [37]:
model1.apply_fill()
for c in model1:
    print(c.mcnp_repr())

2 0 3:-1:2
        IMP:N=0.000 
        U=0 
3 1 -7.800000000000001 -7 -2 -4 6 -9 1 -3 8 -5
        IMP:N=1.000 
        U=0 
4 1 -7.800000000000001 -7 9 -2 -4 1 -10 -3 8 -5
        IMP:N=1.000 
        U=0 
5 0 (10:-6:4) -7 -2 1 -3 8 -5
        IMP:N=1.000 
        U=0 


In [38]:
model1.simplify(min_volume=1.e-3)

Universe 0 simplification has been finished.
0 empty cells were deleted.


In [39]:
for c in model1:
    print(c.mcnp_repr())

2 0 3:-1:2
        IMP:N=0.000 
        U=0 
3 1 -7.800000000000001 -4 6 -9
        IMP:N=1.000 
        U=0 
4 1 -7.800000000000001 9 -4 -10
        IMP:N=1.000 
        U=0 
5 0 (10:-6:4) -2 1 -3
        IMP:N=1.000 
        U=0 


In [40]:
model1.save('test_flattened.i')

In [41]:
surfaces = model1.get_surfaces()
for s in surfaces:
    print(s.mcnp_repr())

4 CZ 3
2 CZ 5
6 PZ -5
1 PZ -10
3 PZ 20
10 PZ 15
9 PZ 5


Now cut a sphere of radius 4 cm from the model1.

In [16]:
sphere = create_surface('SO', 4, name=20)

In [42]:
new_cells = []   # new cells
mask_shape = Shape('S', sphere) # mask shape: only intersections with mask shape will 
                                # be presented in new model
for c in model1:
    nc = c.intersection(mask_shape)  # get intersection cell
    nc = nc.simplify(min_volume=0.1) # simplify intersection cell. 
    new_cells.append(nc)             # append truncated cell to cell list.
new_cells.append(Body(Shape('C', sphere), name=20)) # finally append new body, which is
                                     # the sphere.
new_u = Universe(new_cells)          # And create new universe.

In [43]:
new_u.simplify(min_volume=0.1)

Universe 0 simplification has been finished.
0 empty cells were deleted.


In [44]:
new_u.save('test_cut.i')

In [45]:
cell_1.material()

## Insert cells with material

In [46]:
to_insert = read_mcnp('to_insert.i')

In [47]:
new_cells = [c for c in model1]
ext_cells = []
for add_c in to_insert:
    if add_c.material():
        comp = add_c.shape.complement()
        for i, c in enumerate(new_cells):
            new_cells[i] = c.intersection(comp)
        ext_cells.append(add_c)
new_cells.extend(ext_cells)
comb_u = Universe(new_cells, name_rule='clash')

In [48]:
comb_u.simplify(min_volume=0.01)

Universe 0 simplification has been finished.
0 empty cells were deleted.


In [49]:
comb_u.save('combine.i')

# Material manipulation

In [58]:
comp = Composition(atomic=[('H', 2), ('O', 1)], name=1, comment='Material 1')
print(comp.mcnp_repr())

M1 1000  6.666667e-01
        8000  3.333333e-01


In [51]:
comp_e = comp.expand()   # Get isotope composition
print(comp_e.mcnp_repr())

M1 1001  6.665900e-01
        1002  7.666667e-05
        8016  3.325233e-01
        8017  1.266667e-04
        8018  6.833333e-04


In [52]:
water = Material(comp, density=1.0)

In [53]:
oil = Material(atomic=[('C', 1), ('H', 2)], density=0.8)

In [54]:
mix = Material.mixture((water, 0.4), (oil, 0.6), fraction_type='weight', comment)

In [55]:
print(mix.density)
mix.composition.options['name'] = 3
print(mix.composition.mcnp_repr())

0.8695652173913044
M3 1000  6.666667e-01
        8000  1.138999e-01
        6000  2.194334e-01
